In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go

from analysis import *

In [2]:
dir = "datasets/grocery_data.csv"

d_grocery = load_data(dir)

In [3]:
d_grocery

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001",4
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215",4
3,176560,Google Phone,1,600.00,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001",4
5,176561,Wired Headphones,1,11.99,04/30/19 09:27,"333 8th St, Los Angeles, CA 90001",4
...,...,...,...,...,...,...,...
186845,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001",9
186846,259354,iPhone,1,700.00,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016",9
186847,259355,iPhone,1,700.00,09/23/19 07:39,"220 12th St, San Francisco, CA 94016",9
186848,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016",9


In [5]:
# Apply Function to Column in Dataframe and Add to New Column
d_grocery["City"] = d_grocery["Purchase Address"].apply(lambda x: f"{get_city(x)} {get_state(x)}")

# Calculate Sales which quantity * price each
d_grocery["Sales"] = d_grocery["Quantity Ordered"].astype("int") * d_grocery["Price Each"].astype("float")

bymonth = explore_sales_monthly(d_grocery)
bymonth = bymonth[["Quantity Ordered", "Sales"]]
# bymonth["Month"] = bymonth.index
bymonth.reset_index(level=0, inplace=True)

In [6]:
bymonth

,Month,Quantity Ordered,Sales
0,1,10903,1822256.73
1,2,13449,2202022.42
2,3,17005,2807100.38
3,4,20558,3390670.24
4,5,18667,3152606.75
5,6,15253,2577802.26
6,7,16072,2647775.76
7,8,13448,2244467.88
8,9,13109,2097560.13
9,10,22703,3736726.88


In [7]:
plot_data = [
    go.Scatter(
        x = bymonth["Month"],
        y = bymonth["Quantity Ordered"]
    )
]

plot_layout = go.Layout(
    xaxis={"type": "category"},
    title="Monthly Sales"
)

In [8]:

fig = go.Figure(data = plot_data, layout = plot_layout)
pyoff.iplot(fig)
fig.write_image("graph/fig1.jpeg")

In [27]:
bycity = explore_sales_by_city(d_grocery)
bycity = bycity[["Quantity Ordered", "Sales"]]
bycity.reset_index(level=0, inplace=True)

bycity

,City,Quantity Ordered,Sales
0,Atlanta GA,16602,2795498.58
1,Austin TX,11153,1819581.75
2,Boston MA,22528,3661642.01
3,Dallas TX,16730,2767975.40
4,Los Angeles CA,33289,5452570.80
5,New York City NY,27932,4664317.43
6,Portland ME,2750,449758.27
7,Portland OR,11303,1870732.34
8,San Francisco CA,50239,8262203.91
9,Seattle WA,16553,2747755.48


In [29]:
import plotly.express as px

fig = px.bar(bycity, x="City", y="Sales", title="Sales data by City")
fig.show()

In [61]:
unique_order = d_grocery.query("City=='Dallas TX' and Product=='iPhone'")
unique_order_iphone = unique_order.groupby("Month")["Order ID"].unique().reset_index()
unique_order_iphone["total"] = unique_order_iphone["Order ID"].apply(lambda x: x.shape[0])
unique_order_iphone

,Month,Order ID,total
0,1,"[142261, 142835, 142837, 143240, 143383, 14359...",26
1,2,"[151053, 151216, 151287, 151734, 151759, 15188...",37
2,3,"[162033, 162305, 162391, 162392, 162782, 16314...",35
3,4,"[176732, 176781, 177213, 177316, 177351, 17744...",59
4,5,"[194289, 194594, 194848, 194888, 194935, 19509...",44
5,6,"[210007, 210066, 210181, 210495, 210510, 21073...",47
6,7,"[223785, 223999, 224223, 224282, 226366, 22656...",36
7,8,"[236767, 237976, 238042, 238876, 238915, 23954...",24
8,9,"[248828, 248926, 248985, 250117, 250534, 25127...",28
9,10,"[259580, 259593, 259656, 260230, 260468, 26076...",54


In [62]:
plot_data_iphone = [
    
    go.Scatter(
        x = unique_order_iphone["Month"],
        y = unique_order_iphone["total"]
    )
]

plot_layout_iphone = go.Layout(
    xaxis={"type": "category"},
    title="iPhone Sales by Month on Dallas TX"
)

fig_iphone = go.Figure(data = plot_data_iphone, layout = plot_layout_iphone)
pyoff.iplot(fig_iphone)

In [75]:
active_user_by_states = d_grocery.groupby(["Month", "City"])["Order ID"].unique().reset_index()
active_user_by_states["Order"] = active_user_by_states["Order ID"].apply(lambda x: x.shape[0])
active_user_by_states

,Month,City,Order ID,Order
0,1,Atlanta GA,"[141253, 141261, 141281, 141289, 141296, 14131...",749
1,1,Austin TX,"[300759, 315315, 141238, 141248, 141275, 14127...",510
2,1,Boston MA,"[302393, 306928, 312812, 319308, 141234, 14125...",1000
3,1,Dallas TX,"[297150, 306809, 141266, 141286, 141298, 14129...",755
4,1,Los Angeles CA,"[306655, 141237, 141240, 141242, 141246, 14125...",1472
...,...,...,...,...
115,12,New York City NY,"[295665, 295666, 295667, 295670, 295698, 29570...",3263
116,12,Portland ME,"[295686, 295797, 295862, 296018, 296127, 29615...",307
117,12,Portland OR,"[295671, 295682, 295734, 295757, 295819, 29585...",1283
118,12,San Francisco CA,"[295668, 295675, 295683, 295684, 295685, 29568...",5784


In [78]:
active_user_by_states.query("City == 'Atlanta GA'")

,Month,City,Order ID,Order
0,1,Atlanta GA,"[141253, 141261, 141281, 141289, 141296, 14131...",749
10,2,Atlanta GA,"[150505, 150516, 150538, 150540, 150545, 15054...",904
20,3,Atlanta GA,"[160204, 162016, 162019, 162024, 162029, 16203...",1186
30,4,Atlanta GA,"[176564, 176571, 176582, 176589, 176623, 17662...",1399
40,5,Atlanta GA,"[191156, 194105, 194114, 194123, 194146, 19415...",1262
50,6,Atlanta GA,"[209940, 209948, 209955, 209959, 209976, 21000...",1061
60,7,Atlanta GA,"[222910, 222912, 222913, 222956, 222970, 22298...",1109
70,8,Atlanta GA,"[236685, 236699, 236703, 236714, 236717, 23671...",910
80,9,Atlanta GA,"[248155, 248165, 248173, 248200, 248205, 24825...",881
90,10,Atlanta GA,"[259390, 259394, 259395, 259397, 259406, 25942...",1535


In [81]:
plot_data_compare_city = [

    go.Scatter(
        x = active_user_by_states.query("City == 'Atlanta GA'")["Month"],
        y = active_user_by_states.query("City == 'Atlanta GA'")["Order"],
        name = "Atlanta GA"
    ),
    go.Scatter(
        x = active_user_by_states.query("City == 'Dallas TX'")["Month"],
        y = active_user_by_states.query("City == 'Dallas TX'")["Order"],
        name = "Dallas TX"
    ),
    go.Scatter(
        x = active_user_by_states.query("City == 'Boston MA'")["Month"],
        y = active_user_by_states.query("City == 'Boston MA'")["Order"],
        name = "Boston MA"
    )
]

plot_layout_compare_city = go.Layout(
    xaxis = {"type": "category"},
    title = "Order on Cities by Month"
)

fig_compare_city = go.Figure(data = plot_data_compare_city, layout = plot_layout_compare_city)
pyoff.iplot(fig_compare_city)
